In [13]:
import pandas as pd
import sqlite3

# Connect to the database (creates the file if it doesn't exist)
temp_path = './database.db'
connection = sqlite3.connect(temp_path)


1. Looks like a snowflake schema

2. How many different Customers are there?

In [14]:
# Query the sqlite_master table for table names
query = 'SELECT COUNT(DISTINCT CustomerId) AS UniqueCustomerCount FROM customers;'
temp_df = pd.read_sql_query(query, connection)
temp_df
# temp_df['CustomerId'].unique()

DatabaseError: Execution failed on sql 'SELECT COUNT(DISTINCT CustomerId) AS UniqueCustomerCount FROM customers;': no such table: customers

3. Which Genre has the shortest average track length, in minutes?

In [ ]:
# Query the sqlite_master table for table names
query = "SELECT GenreId, Milliseconds FROM tracks;"
temp_df = pd.read_sql_query(query, connection)
temp_df = temp_df.groupby('GenreId')['Milliseconds'].mean()
temp_df = temp_df.apply(lambda x:x/1000/60)
temp_df = temp_df.sort_values().reset_index()
query = "SELECT * FROM genres;"
temp_df1 = pd.read_sql_query(query, connection)
temp_df.merge(temp_df1, left_on='GenreId', right_on='GenreId', how='left').iloc[0]

GenreId                     5
Milliseconds         2.244058
Name            Rock And Roll
Name: 0, dtype: object

4. Using the ‘playlists’ table, can you tell me the name of the artist that appears the
most in playlists?

In [ ]:
# Query the sqlite_master table for table names
query = "SELECT * FROM playlist_track;"
temp_df = pd.read_sql_query(query, connection)
query = "SELECT TrackId, AlbumId FROM tracks;"
temp_df1 = pd.read_sql_query(query, connection)
query = "SELECT AlbumId, ArtistId FROM albums;"
temp_df2 = pd.read_sql_query(query, connection)
query = "SELECT * FROM artists;"
temp_df3 = pd.read_sql_query(query, connection)

In [ ]:
merge_df = temp_df.merge(temp_df1, left_on='TrackId', right_on='TrackId', how='left')
merge_df = merge_df.merge(temp_df2, left_on='AlbumId', right_on='AlbumId', how='left')
res_df = merge_df.groupby('ArtistId').size().reset_index().sort_values(by=0)
res_df = res_df.merge(temp_df3, left_on='ArtistId', right_on='ArtistId')
res_df.iloc[-1]

ArtistId             90
0                   516
Name        Iron Maiden
Name: 203, dtype: object

What is the count of Customers who have bought more than $40 of tracks?

In [ ]:
# Query the sqlite_master table for table names
query = "SELECT InvoiceId, CustomerId FROM invoices;"
temp_df = pd.read_sql_query(query, connection)
query = "SELECT InvoiceLineId, InvoiceId, UnitPrice, Quantity FROM invoice_items;"
temp_df1 = pd.read_sql_query(query, connection)

In [ ]:
merge_df = temp_df.merge(temp_df1, left_on='InvoiceId', right_on='InvoiceId', how='left')
merge_df['Revenue'] = merge_df['UnitPrice']*merge_df['Quantity']
res_df = merge_df.groupby('CustomerId')['Revenue'].sum()
res_df = res_df[res_df > 40]
res_df.size

14

Which Key did you use the most?

CustomerId

In [ ]:
# Close the connection
connection.close()